# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [2]:
print(
    "The dataset collected is related to 17 marketing campaigns \n "
    "that occurred between May 2008 and November 2010, \n"
    "corresponding to a total of 79,354 contacts.\n"
    "Source: CRISP-DM-BANK.pdf (Moro et al., 2014) \n"
)






The dataset collected is related to 17 marketing campaigns 
 that occurred between May 2008 and November 2010, 
corresponding to a total of 79,354 contacts.
Source: CRISP-DM-BANK.pdf (Moro et al., 2014) 



### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [5]:
import pandas as pd
bank_df = pd.read_csv("data/bank-additional-full.csv")
bank_df.head()

,"age;""job"";""marital"";""education"";""default"";""housing"";""loan"";""contact"";""month"";""day_of_week"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""emp.var.rate"";""cons.price.idx"";""cons.conf.idx"";""euribor3m"";""nr.employed"";""y"""
0,"56;""housemaid"";""married"";""basic.4y"";""no"";""no"";..."
1,"57;""services"";""married"";""high.school"";""unknown..."
2,"37;""services"";""married"";""high.school"";""no"";""ye..."
3,"40;""admin."";""married"";""basic.6y"";""no"";""no"";""no..."
4,"56;""services"";""married"";""high.school"";""no"";""no..."


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [15]:
import pandas as pd

# Load the data
bank_df = pd.read_csv("data/bank-additional-full.csv", sep=';')  # Added sep=';' as bank data typically uses semicolons
print(bank_df.columns.tolist())  # Print column names to verify what's available
bank_df.head()

# Step 3: Define categorical columns based on actual column names
# Adjust these column names to match what was printed above
categorical_cols = ['job','marital','education','default','housing','loan',
                    'contact','month','day_of_week','poutcome','y']

# Step 4: Convert categorical columns to 'category' type
for col in categorical_cols:
    if col in bank_df.columns:  # Check if column exists before converting
        bank_df[col] = bank_df[col].astype('category')
    else:
        print(f"Warning: Column '{col}' not found in the dataset")

# Step 5: Check for 'unknown' values in key categorical features
print("Counts of 'unknown' values in selected categorical columns:\n")
for col in ['job','marital','education','default','housing','loan']:
    if col in bank_df.columns:  # Check if column exists before accessing
        print(f"{col}:")
        print(bank_df[col].value_counts())
        print("-"*40)
    else:
        print(f"Warning: Column '{col}' not found in the dataset")

# Step 6: Quick summary of numeric columns
numeric_cols = ['age','duration','campaign','pdays','previous',
                'emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']

# Filter to only include columns that exist in the dataset
valid_numeric_cols = [col for col in numeric_cols if col in bank_df.columns]

print("\nNumeric columns summary:\n")
print(bank_df[valid_numeric_cols].describe())

# Step 7: Suggest dropping 'duration' for realistic predictive modeling
print("\nNote: 'duration' should be dropped if predicting subscription before the call occurs.")

['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']
Counts of 'unknown' values in selected categorical columns:

job:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64
----------------------------------------
marital:
marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64
----------------------------------------
education:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y            

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [21]:
print("Business Objective:\n"
      "Predict whether a bank client will subscribe to a term deposit (y = yes/no) "
      "using the data provided in the sample , age , job , marital status , etc.\n"
      "The goal is to help the bank optimize marketing efforts, reduce costs of unsuccessful calls, "
      "and increase subscription success rates.")

Business Objective:
Predict whether a bank client will subscribe to a term deposit (y = yes/no) using the data provided in the sample , age , job , marital status , etc.
The goal is to help the bank optimize marketing efforts, reduce costs of unsuccessful calls, and increase subscription success rates.


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [23]:
# Select bank client features (first 7)
X = bank_df[['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']]

# Target variable
y = bank_df['y']
categorical_cols = ['job','marital','education','default','housing','loan']
# One-hot encode categorical columns
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
# Encode target: 'yes' -> 1, 'no' -> 0
y_encoded = y.map({'yes': 1, 'no': 0})
print("Features shape:", X_encoded.shape)
print("Target shape:", y_encoded.shape)
print("\nFirst 5 rows of encoded features:")
print(X_encoded.head())


Features shape: (41188, 28)
Target shape: (41188,)

First 5 rows of encoded features:
   age  job_blue-collar  job_entrepreneur  job_housemaid  job_management  \
0   56            False             False           True           False   
1   57            False             False          False           False   
2   37            False             False          False           False   
3   40            False             False          False           False   
4   56            False             False          False           False   

   job_retired  job_self-employed  job_services  job_student  job_technician  \
0        False              False         False        False           False   
1        False              False          True        False           False   
2        False              False          True        False           False   
3        False              False         False        False           False   
4        False              False          True        Fa

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [25]:
from sklearn.model_selection import train_test_split
# Split the dataset into train and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (28831, 28)
X_test shape: (12357, 28)
y_train shape: (28831,)
y_test shape: (12357,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [27]:
# Count target classes
class_counts = y_encoded.value_counts()
print(class_counts)

# Calculate proportion of majority class
baseline_accuracy = class_counts.max() / class_counts.sum()
print(f"Baseline accuracy (majority class): {baseline_accuracy:.4f}")


y
0    36548
1     4640
Name: count, dtype: int64
Baseline accuracy (majority class): 0.8873


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Initialize Logistic Regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Test Accuracy: 0.8874

Confusion Matrix:
[[10965     0]
 [ 1392     0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     10965
           1       0.00      0.00      0.00      1392

    accuracy                           0.89     12357
   macro avg       0.44      0.50      0.47     12357
weighted avg       0.79      0.89      0.83     12357



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Problem 9: Score the Model

What is the accuracy of your model?

In [31]:
from sklearn.metrics import accuracy_score

# Calculate test accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8874


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [37]:
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define models with default settings
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(random_state=42)
}

# List to store results
results = []

# Loop through models
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)             # Train the model
    train_time = time.time() - start_time   # Measure fit time
    
    # Predict on train and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Compute accuracies
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    # Append results
    results.append({
        "Model": name,
        "Train Time (s)": train_time,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Test Accuracy", ascending=False)
results_df.reset_index(drop=True, inplace=True)

# Display results
results_df


,Model,Train Time (s),Train Accuracy,Test Accuracy
0,Logistic Regression,0.317924,0.887343,0.887351
1,SVM,3.447798,0.887343,0.887351
2,KNN,0.002605,0.890985,0.875293
3,Decision Tree,0.041436,0.918837,0.863883


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [75]:
# ==============================
# PROOF: Feature Engineering, Hyperparameter Tuning, and Metrics
# Fully robust version
# ==============================

import pandas as pd
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# ------------------------------
# 1️⃣ Load Dataset
# ------------------------------
file_path = "data/bank-additional-full.csv"
bank_df = pd.read_csv(file_path, sep=';')

# Inspect columns
print("Columns in dataset:", bank_df.columns.tolist())

# ------------------------------
# 2️⃣ Detect Target Column Automatically
# ------------------------------
target_col = None
for col in bank_df.columns:
    if set(bank_df[col].dropna().unique()) == {'yes','no'}:
        target_col = col
        break

if target_col is None:
    raise ValueError("No column with 'yes'/'no' values found in dataset!")

print("Detected target column:", target_col)

# ------------------------------
# 3️⃣ Feature Engineering
# ------------------------------
# Create total_loans if possible
loan_cols = ['housing','loan','default']
existing_loan_cols = [col for col in loan_cols if col in bank_df.columns]

if existing_loan_cols:
    bank_df['total_loans'] = bank_df[existing_loan_cols].apply(
        lambda x: sum([1 if v=='yes' else 0 for v in x]), axis=1
    )
else:
    bank_df['total_loans'] = 0

# ------------------------------
# 4️⃣ Detect Numeric and Categorical Columns Automatically
# ------------------------------
numeric_cols = bank_df.select_dtypes(include=['int64','float64']).columns.tolist()
# Include total_loans if it exists
if 'total_loans' in bank_df.columns and 'total_loans' not in numeric_cols:
    numeric_cols.append('total_loans')

# Exclude target if numeric
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

categorical_cols = bank_df.select_dtypes(include=['object']).columns.tolist()
# Exclude target from categorical
if target_col in categorical_cols:
    categorical_cols.remove(target_col)

# ------------------------------
# 5️⃣ One-hot Encoding
# ------------------------------
X = pd.get_dummies(bank_df[categorical_cols + numeric_cols], drop_first=True)

# Map target to 0/1
y = bank_df[target_col].map({'yes':1,'no':0})

# ------------------------------
# 6️⃣ Train-Test Split
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standardize numeric features for KNN/SVM
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


# Check the target variable format and convert if necessary
# If y contains 'yes'/'no' values, convert them to 1/0
if isinstance(y_train.iloc[0], str):
    # Convert 'yes'/'no' or other string labels to 0/1
    y_train = y_train.map({'yes': 1, 'no': 0}) if 'yes' in y_train.unique() else y_train
    y_test = y_test.map({'yes': 1, 'no': 0}) if 'yes' in y_test.unique() else y_test
    
# ------------------------------
# 7️⃣ Hyperparameter Tuning
# ------------------------------
# Decision Tree
dt_param_grid = {'max_depth':[3,5,7,10,None],
                 'min_samples_split':[2,5,10],
                 'min_samples_leaf':[1,2,4]}
grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_dt.fit(X_train, y_train)

# KNN
knn_param_grid = {'n_neighbors':[3,5,7,9], 'weights':['uniform','distance']}
grid_knn = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_knn.fit(X_train, y_train)

# Logistic Regression
lr_param_grid = {'C':[0.01,0.1,1,10], 'penalty':['l2']}
grid_lr = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), lr_param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_lr.fit(X_train, y_train)

# SVM
svm_param_grid = {'C':[0.1,1,10], 'kernel':['linear','rbf'], 'gamma':['scale','auto']}
grid_svm = GridSearchCV(SVC(random_state=42, probability=True), svm_param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_svm.fit(X_train, y_train)

# ------------------------------
# 8️⃣ Evaluate Metrics
# ------------------------------
models = {
    'Decision Tree': grid_dt.best_estimator_,
    'KNN': grid_knn.best_estimator_,
    'Logistic Regression': grid_lr.best_estimator_,
    'SVM': grid_svm.best_estimator_
}

results = []

for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    results.append({
        'Model': name,
        'Train Time (s)': round(train_time,3),
        'Train Accuracy': round(accuracy_score(y_train, y_train_pred),4),
        'Test Accuracy': round(accuracy_score(y_test, y_test_pred),4)
    })

results_df = pd.DataFrame(results).sort_values(by='Test Accuracy', ascending=False)
results_df.reset_index(drop=True, inplace=True)

print("\nModel Comparison Results:\n", results_df)


Columns in dataset: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']
Detected target column: y

Model Comparison Results:
                  Model  Train Time (s)  Train Accuracy  Test Accuracy
0  Logistic Regression           0.136          0.9108         0.9121
1        Decision Tree           0.029          0.9090         0.9098
2                  SVM          52.940          0.9546         0.9097
3                  KNN           0.009          0.9407         0.9002


In [2]:
# ==============================
# Predict Customers Likely to Subscribe
# UCI Bank Marketing Dataset
# ==============================

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# ------------------------------
# 1️⃣ Load dataset (IMPORTANT: semicolon separator)
# ------------------------------
file_path = "data/bank-additional-full.csv"

bank_df = pd.read_csv(file_path, sep=";")
print("Dataset loaded successfully")
print(bank_df.head())

# ------------------------------
# 2️⃣ Separate features and target
# ------------------------------
X = bank_df.drop(columns=["y"])
y = bank_df["y"].map({"yes": 1, "no": 0})

# ------------------------------
# 3️⃣ One-hot encode categorical features
# ------------------------------
X_encoded = pd.get_dummies(X, drop_first=True)

# ------------------------------
# 4️⃣ Train-test split
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# ------------------------------
# 5️⃣ Scale features
# ------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------
# 6️⃣ Train Logistic Regression model
# ------------------------------
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

print("Model trained successfully")

# ------------------------------
# 7️⃣ Predict subscription probabilities
# ------------------------------
subscription_probabilities = model.predict_proba(X_test_scaled)[:, 1]

# ------------------------------
# 8️⃣ Select likely subscribers (threshold = 70%)
# ------------------------------
threshold = 0.70
likely_subscribers_idx = X_test.index[subscription_probabilities >= threshold]

likely_subscribers = bank_df.loc[likely_subscribers_idx]

print(f"\nNumber of likely subscribers (prob ≥ {threshold}):", len(likely_subscribers))

# ------------------------------
# 9️⃣ View sample likely subscribers
# ------------------------------
print("\nSample likely subscribers:")
print(likely_subscribers.head())

# ------------------------------
# 🔟 (Optional) Save results
# ------------------------------
likely_subscribers.to_csv("likely_subscribers.csv", index=False)
print("\nFile saved as likely_subscribers.csv")

Dataset loaded successfully
   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf

##### Questions